Work on improving the tree traversel I had previously implemented.

In [1]:
%run ../scripts/notebook_settings_lean.py
import tskit

Loading ts and poplabels

In [2]:
ts = tskit.load("../results/All_Samples_relate/chromhapX.trees")

In [3]:
poplabels = pd.read_csv("../data/pops/haploid_x_8cluster.sample", sep=" ")

In [4]:
# Version which counts number of samples and notes when 50 % has coalesced.
def coalescence_ordering(tree, IDs, sample_counts):
    df_list = []
    for i in IDs:
        pop_list = []
        gen_list = []
        current_node = i
        while tree.depth(current_node) > 0:
            # Find parent node
            parent_node = tree.parent(current_node)
            c_samples = pd.Series([x for x in tree.samples(current_node)])
            c_sample_counts = c_samples.map(i_mapping).value_counts()
            # If pop is not already added to list, add pop and note coal time when it surpasses 50 %
            for p in c_sample_counts.index:
                if p  not in pop_list and c_sample_counts[p] > sample_counts[p]/2:
                    pop_list.append(p)
                    gen_list.append(tree.time(current_node))
            current_node = parent_node
        d = {"ID": i, "sites": tree.num_sites, "span": tree.span, "start": tree.interval[0]}
        for i in range(len(pop_list)):
            d["coal_{}".format(i)] = pop_list[i]
        for i in range(len(gen_list)):
            d["coal_date_{}".format(i)] = gen_list[i]
        df_list.append(pd.DataFrame(d, index=[i]))
    return pd.concat(df_list)

In [5]:
i_mapping = {}
for i, row in poplabels.iterrows():
    i_mapping[i] = row.GROUP
poplabel_counts = poplabels["GROUP"].value_counts().to_dict()
highest_ID = poplabels.index.max()+1

In [9]:
#%%timeit -n 10
c = 0
ID_list = list(i_mapping.keys())[10:12]
sample_counts = poplabels["GROUP"].value_counts()
for tree in ts.trees():
    if c >= 100:
        break
    df = coalescence_ordering(tree, ID_list, sample_counts)
    c += 1

In [14]:
%%timeit -n 10
c = 0
IDs = list(i_mapping.keys())[10:12]
for tree in ts.trees():
    if c >= 1:
        break
    c += 1
    df_list = []
    for i in IDs:
        pop_list = []
        gen_list = []
        current_node = i
        while tree.depth(current_node) > 0:
            # Find parent node
            parent_node = tree.parent(current_node)
            c_samples = pd.Series([x for x in tree.samples(current_node)])
            c_sample_counts = c_samples.map(i_mapping).value_counts()
            # If pop is not already added to list, add pop and note coal time when it surpasses 50 %
            #for p in c_sample_counts.index:
            #    if p  not in pop_list and c_sample_counts[p] > sample_counts[p]/2:
            #        pop_list.append(p)
            #        gen_list.append(tree.time(current_node))
            current_node = parent_node

980 ms ± 41.9 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [15]:
%%timeit -n 10
c = 0
IDs = list(i_mapping.keys())[10:12]
for tree in ts.trees():
    if c >= 1:
        break
    c += 1
    df_list = []
    for i in IDs:
        pop_list = []
        gen_list = []
        current_node = i
        while tree.depth(current_node) > 0:
            # Find parent node
            parent_node = tree.parent(current_node)
            c_samples = pd.Series([x for x in tree.samples(current_node)])
            c_sample_counts = c_samples.map(i_mapping).value_counts()
            # If pop is not already added to list, add pop and note coal time when it surpasses 50 %
            for p in c_sample_counts.index:
                if p  not in pop_list and c_sample_counts[p] > sample_counts[p]/2:
                    pop_list.append(p)
                    gen_list.append(tree.time(current_node))
            current_node = parent_node

866 ms ± 61.1 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [17]:
for u in tree.samples():
    path = []
    v = u
    while v != tskit.NULL:
        path.append(v)
        v = tree.parent(v)
    print(u, "->", path)

182 -> [182, 564, 642]
178 -> [178, 563, 564, 642]
176 -> [176, 323, 563, 564, 642]
179 -> [179, 323, 563, 564, 642]
186 -> [186, 407, 487, 638, 641, 642]
188 -> [188, 407, 487, 638, 641, 642]
208 -> [208, 430, 487, 638, 641, 642]
194 -> [194, 375, 430, 487, 638, 641, 642]
204 -> [204, 375, 430, 487, 638, 641, 642]
1 -> [1, 517, 611, 631, 638, 641, 642]
17 -> [17, 411, 517, 611, 631, 638, 641, 642]
314 -> [314, 411, 517, 611, 631, 638, 641, 642]
130 -> [130, 379, 606, 611, 631, 638, 641, 642]
132 -> [132, 379, 606, 611, 631, 638, 641, 642]
112 -> [112, 520, 606, 611, 631, 638, 641, 642]
4 -> [4, 410, 520, 606, 611, 631, 638, 641, 642]
104 -> [104, 410, 520, 606, 611, 631, 638, 641, 642]
197 -> [197, 335, 616, 629, 631, 638, 641, 642]
236 -> [236, 335, 616, 629, 631, 638, 641, 642]
223 -> [223, 485, 616, 629, 631, 638, 641, 642]
243 -> [243, 464, 485, 616, 629, 631, 638, 641, 642]
230 -> [230, 364, 464, 485, 616, 629, 631, 638, 641, 642]
232 -> [232, 364, 464, 485, 616, 629, 631, 638, 6

In [21]:
tree.postorder()

array([182, 178, 176, 179, 323, 563, 564, 186, 188, 407, 208, 194, 204,
       375, 430, 487,   1,  17, 314, 411, 517, 130, 132, 379, 112,   4,
       104, 410, 520, 606, 611, 197, 236, 335, 223, 243, 230, 232, 364,
       464, 485, 616,  97,  90,  98,  95,  96, 334, 469, 470, 581,  93,
        99,  86, 101, 361, 431, 438,  87, 100, 333,  92,  94, 408, 503,
       594, 599,  89,  88,  85,  91, 329, 435, 600, 603, 218, 237, 350,
       240, 228, 217, 219, 238, 390, 471, 474, 478, 596, 106,   7, 107,
       342, 442, 213, 220, 222, 405, 539, 212,  19,  22, 169, 170, 394,
       443, 513, 514, 226, 216, 229, 392, 477, 183, 203, 372, 190, 321,
       209, 193, 200, 351, 467, 468, 546, 552, 234, 231, 239, 330, 488,
       225, 227, 322, 221, 108, 224, 244, 235, 215, 241, 349, 461, 462,
       466, 504, 505, 548, 570, 579, 580, 617, 618, 242, 127, 137, 139,
       341, 436, 557, 185, 214, 233, 382, 502, 140, 121, 123, 377, 428,
        23,  15, 166, 353, 444,  14,  64,  60, 168, 378, 460, 51

In [34]:
for n in tree.timeasc()[highest_ID:]:
    c_samples = (pd.Series([x for x in tree.samples(n)]))
    c_sample_counts = c_samples.map(i_mapping).value_counts()
    print(c_sample_counts)

Tanzanian_Olive    2
Name: count, dtype: int64
Tanzanian_Olive    2
Name: count, dtype: int64
Tanzanian_Olive    2
Name: count, dtype: int64
Papio    2
Name: count, dtype: int64
Tanzanian_Olive    2
Name: count, dtype: int64
Kindae    2
Name: count, dtype: int64
Kindae    2
Name: count, dtype: int64
Kindae    2
Name: count, dtype: int64
Tanzanian_Olive    2
Name: count, dtype: int64
Tanzanian_Olive    3
Name: count, dtype: int64
Tanzanian_Olive    3
Name: count, dtype: int64
Ethiopian_Olive    2
Name: count, dtype: int64
Tanzanian_Olive    2
Name: count, dtype: int64
Papio    3
Name: count, dtype: int64
Hamadryas    2
Name: count, dtype: int64
Tanzanian_Olive    2
Name: count, dtype: int64
Eastern_Yellow    2
Name: count, dtype: int64
Tanzanian_Olive    2
Name: count, dtype: int64
Hamadryas    2
Name: count, dtype: int64
Tanzanian_Olive    3
Name: count, dtype: int64
Eastern_Yellow    2
Name: count, dtype: int64
Hamadryas    2
Name: count, dtype: int64
Tanzanian_Olive    2
Name: count,

Chacma    3
Name: count, dtype: int64
Eastern_Yellow     69
Kindae             36
Tanzanian_Olive    18
Western_Yellow     11
Chacma              3
Name: count, dtype: int64
Chacma    4
Name: count, dtype: int64
Eastern_Yellow     69
Kindae             36
Hamadryas          18
Tanzanian_Olive    18
Western_Yellow     11
Chacma              3
Name: count, dtype: int64
Tanzanian_Olive    92
Ethiopian_Olive    33
Papio              17
Hamadryas          14
Western_Yellow      2
Kenyan_Olive        1
Name: count, dtype: int64
Eastern_Yellow     73
Kindae             36
Hamadryas          18
Tanzanian_Olive    18
Western_Yellow     11
Chacma              3
Name: count, dtype: int64
Tanzanian_Olive    110
Eastern_Yellow      73
Kindae              36
Ethiopian_Olive     33
Hamadryas           32
Papio               17
Western_Yellow      13
Chacma               3
Kenyan_Olive         1
Name: count, dtype: int64
Tanzanian_Olive    110
Eastern_Yellow      73
Kindae              36
Ethiopian_Ol